In [1]:
import numpy as np 
import pandas as pd 
import os
from torchvision import transforms, utils, models, datasets
import torch
from torch.utils.data import DataLoader
import torchvision.transforms.functional as F

from utils.engine import train_one_epoch, evaluate
from utils import utils

import torch
from utils import transforms as T

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

from ADEdataset import ADEDataset

In [2]:
compose = transforms.Compose([transforms.PILToTensor(),
                                transforms.ConvertImageDtype(torch.float),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])


def voc_transform_ing(img):
    
    target_anno = img["annotation"]
    w, h, d = list(map(int, target_anno['size'].values()))

    target = {"boxes":[], "labels":[],'area':[]}
    for obj in target_anno['object']:
        box = list(map(int,obj['bndbox'].values()))
        target["boxes"].append(box)
        target['labels'].append(classes.index(obj["name"]))
        
        area = (box[3] - box[1]) * (box[2] - box[0])
        target["area"].append(area)    

    iscrowd = torch.zeros((len(target["boxes"]),), dtype=torch.int64)

    target["image_id"] = torch.Tensor([int(target_anno['filename'][:-4])])
    target["boxes"] = torch.FloatTensor(target["boxes"])
    target["labels"] = torch.LongTensor(target["labels"])
    target["area"] = torch.FloatTensor(target["area"])
    target["iscrowd"] = iscrowd
    target["masks"] = torch.zeros((len(target["boxes"]),w, h), dtype=torch.float32)

    return target

In [3]:
classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 
           'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 
           'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']


path_dir_trainval = "../hw4_dataset/VOCTrainVal"
path_dir_test = "../hw4_dataset/VOCtest"

voctrainval_ds = datasets.VOCDetection(root=path_dir_trainval, year="2007", image_set="trainval", transform=compose, target_transform=voc_transform_ing)
voc_trainval = DataLoader(voctrainval_ds, batch_size=8, collate_fn = utils.collate_fn, shuffle=True)

voctest_ds = datasets.VOCDetection(root=path_dir_test, year="2007", image_set="test", transform=compose,target_transform=voc_transform_ing)
voc_test = DataLoader(voctest_ds, batch_size=8, collate_fn = utils.collate_fn, shuffle=True)


In [4]:
print(f"trainval dataset :{len(voctrainval_ds)} images")
print(f"test dataset :{len(voctest_ds)} images")
print(f"the meta data in image:{voctrainval_ds[0][1].keys()}")

trainval dataset :5011 images
test dataset :4952 images
the meta data in image:dict_keys(['boxes', 'labels', 'area', 'image_id', 'iscrowd', 'masks'])


In [5]:
root = "../ADE20K"
train_data = ADEDataset(root,"train.txt", compose)
ade_train = torch.utils.data.DataLoader(
                                train_data, batch_size=2, shuffle=True, num_workers=4,
                                collate_fn=utils.collate_fn)
val_data = ADEDataset(root,"val.txt", compose)
ade_val = torch.utils.data.DataLoader(
                                val_data, batch_size=2, shuffle=True, num_workers=4,
                                collate_fn=utils.collate_fn)
test_data = ADEDataset(root,"test.txt", compose)
ade_test = torch.utils.data.DataLoader(
                                test_data, batch_size=2, shuffle=True, num_workers=4,
                                collate_fn=utils.collate_fn)


In [6]:
print(f"train dataset :{len(train_data)} images")
print(f"val dataset :{len(val_data)} images")
print(f"test dataset :{len(test_data)} images")
# print(f"the meta data in image:{voctrainval_ds[0][1].keys()}")

train dataset :1400 images
val dataset :200 images
test dataset :400 images


In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

lr = 0.001
batch_size = 8
weight_decay=1e-5
num_classes = 150
#model = get_model_instance_segmentation(num_classes).to(device)
model = models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(score_thresh=0.5, weights_backbone=True, num_classes=num_classes).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr = lr, weight_decay=weight_decay)

/home/re6114056/miniconda3/envs/image/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights_backbone' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights_backbone=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights_backbone=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# voc_trainval, voc_test, ade_train, ade_val, ade_test

In [9]:
num_epochs = 1

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
round = ["voc",'ade']
for epoch in range(num_epochs):
    
    train_one_epoch(model, optimizer, voc_trainval, device, epoch=epoch, run_dataset='voc', 
                 print_freq=10, scaler=None)
    train_one_epoch(model, optimizer, ade_train, device, epoch=epoch, run_dataset='ade', 
                    print_freq=10, scaler=None)
    
    lr_scheduler.step()
        # evaluate on the test dataset3
    evaluate(model, voc_test, device=device)
    evaluate(model, ade_test, device=device)

Epoch: [0]  [  0/627]  eta: 0:24:51  lr: 0.000003  loss: 6.0458 (6.0458)  loss_classifier: 5.0889 (5.0889)  loss_box_reg: 0.2382 (0.2382)  loss_objectness: 0.6961 (0.6961)  loss_rpn_box_reg: 0.0225 (0.0225)  time: 2.3784  data: 0.0620  max mem: 76
Epoch: [0]  [ 10/627]  eta: 0:04:41  lr: 0.000019  loss: 6.0458 (6.0280)  loss_classifier: 5.0747 (5.0774)  loss_box_reg: 0.2433 (0.2224)  loss_objectness: 0.6907 (0.6908)  loss_rpn_box_reg: 0.0273 (0.0374)  time: 0.4563  data: 0.0615  max mem: 76
Epoch: [0]  [ 20/627]  eta: 0:03:33  lr: 0.000035  loss: 5.9859 (6.0049)  loss_classifier: 5.0620 (5.0672)  loss_box_reg: 0.2314 (0.2209)  loss_objectness: 0.6874 (0.6844)  loss_rpn_box_reg: 0.0273 (0.0324)  time: 0.2509  data: 0.0593  max mem: 76
Epoch: [0]  [ 30/627]  eta: 0:03:09  lr: 0.000050  loss: 5.9999 (6.0238)  loss_classifier: 5.0620 (5.0961)  loss_box_reg: 0.2390 (0.2373)  loss_objectness: 0.6598 (0.6592)  loss_rpn_box_reg: 0.0229 (0.0312)  time: 0.2400  data: 0.0578  max mem: 76
Epoch: [